In [1]:
# import Pkg;
# Pkg.add("POMDPSimulators")
# Pkg.add("POMDPs")
# Pkg.add("POMDPTools")
# Pkg.add("QuickPOMDPs")
# Pkg.add("MCTS")
# Pkg.add("DiscreteValueIteration")
# Pkg.add("POMDPModels")
# Pkg.add("POMDPPolicies")
# Pkg.add("Distributions")
# Pkg.add("Combinatorics")
# Pkg.add("StaticArrays")
# Pkg.add("D3Trees")
# Pkg.add("Random")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("TickTock")



In [2]:
# import Pkg; Pkg.add("DataFrames")

In [3]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [4]:
global Number_level = 4;
global Action_num = 4;
gamma=0.95

States=Array((1:Number_level))
Observations=Array((1:Number_level))

4-element Vector{Int64}:
 1
 2
 3
 4

In [5]:
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]

4×4 Matrix{Float64}:
 0.8  0.2  0.0  0.0
 0.1  0.8  0.1  0.0
 0.0  0.2  0.8  0.0
 0.0  0.0  0.0  1.0

In [6]:
T=zeros(Number_level,Number_level,3,100);

# Load transition probability matrix for each component
for i in 1:100
    fullname = "C:/Users/hhu98/OneDrive - UW-Madison/POMDP/code/Heterogeneous system/hetero_tm/tm"*string(i);
   
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    # the last term in argument of T, which is written as 'i' here,
    # is the index of component in the heterogeneous system
    T[:,:,1,i].=Transition_matrix; # a=1: do not replace
    T[:,:,2,i].=Transition_matrix; # a=2: corrective replacement
    for j in 1:Number_level
        T[j,:,3,i].=Transition_matrix[1,:]; # a=3: preventive replacement
        end
    # T[Number_level,:,1,i].=Transition_matrix[1,:]
    T[Number_level,:,1,i].=0.0
    T[Number_level,Number_level,1,i]=1.0
    T[Number_level,:,2,i].=Transition_matrix[1,:]

end

┌ Warning: This file was written in a newer version of the JLD2 file format.
│ Please consider updating JLD2.
└ @ JLD2 C:\Users\hhu98\.julia\packages\JLD2\VWinU\src\file_header.jl:39
┌ Warning: This file was written with a different version of JLD2 that may not be compatible.
│ Attempting to load data.
└ @ JLD2 C:\Users\hhu98\.julia\packages\JLD2\VWinU\src\file_header.jl:43


In [7]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [8]:
SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [ ]:
Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstates
Initialstate = SparseCat(States, Initialstates)

Initialobs = SparseCat(Observations, Observation_matrix[1,:])

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [12]:
function create_pomdps()

    Unit_pomdps = Dict{Int,POMDP}()
    
    for i in 1:NumberUnits
    
     Unit_pomdps[i]= QuickPOMDP(
        states=States,
        actions=Array((1:Action_num)),
        observations = Observations,
        initialstate = Initialstate,
        initialobs = Initialobs,
        discount = gamma,
    
        transition = function(s,a)
            if a == 1 # NO REPAIR, NO SYSTEM FAILURE
                return SparseCat(States, T[s,:,1,i])
            end
            if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
                if s == Number_level
                    s = 1
                    return SparseCat(States, T[s,:,2,i])
                else
                    return SparseCat(States, T[s,:,2,i])
                end
            end
            if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE
                if s==Number_level
                    s=1
                    return SparseCat(States, T[s,:,3,i])
                else
                    s = 1
                    return SparseCat(States, T[s,:,3,i])
                end
            end
            if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
                if s==Number_level
                    s=1
                    return SparseCat(States, T[s,:,3,i])
                else
                    s = 1
                    return SparseCat(States, T[s,:,3,i])
                end
            end
        end,
    
        observation = function(a,s)
            return SparseCat(Observations, Observation_matrix[s,:])
        end,
            
        reward = function (s, a)
            if a == 1 # NO REPAIR, NO SYSTEM FAILURE
                if s == Number_level
                    return system_penalty/(limit)
                else
                    return normal_operation
                end
            end
            if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM, NO SYSTEM FAILURE PENALTY
                if s == Number_level
                    return failure_penalty +  setup_cost/NumberUnits + normal_operation
                else
                    # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                    return normal_operation +  setup_cost/NumberUnits
                end
            end
            if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE PENALTY
                if s==Number_level
                    return failure_penalty +  setup_cost/NumberUnits + normal_operation
                else
                    # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                    return maintenance_penalty +  setup_cost/NumberUnits+ normal_operation 
                end
            end
            if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
                if s==Number_level
                    return failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit)
                else
                    return maintenance_penalty +  setup_cost/NumberUnits + normal_operation
                end
            end
        end,
        
    )
    end
    return Unit_pomdps
    end

create_pomdps (generic function with 1 method)

In [13]:
function generative(s,a,i,rng=23) # calculate TRUE COMPONENT-LEVEL reward r_i(s_i,a_i) and get evolving states
    if a==1 # No maintenance in the system, no system failure
        crd=Categorical(T[s,:,1,i])
        s=rand(crd)
        r=normal_operation
        obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
        obs = rand(obs_distribution)
        return (sp=s,r=r,o=obs)
    end

    if a==2 # Maintenance occurs in the system, no system failure, corrective maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,2,i])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,2,i])
            s = rand(crd)
            r = normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==3 # Maintenance occurs in the system, no system failure, preventive maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,3,i])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,3,i])
            s = rand(crd)
            r = maintenance_penalty+normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==4 # maintenance in system, system failure occurs, preventive replacement for the unit
        if s==Number_level
            crd = Categorical(T[1,:,3,i])
            s = rand(crd)
            r=failure_penalty+ normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            r = maintenance_penalty+normal_operation
            crd = Categorical(T[1,:,3,i])
            s = rand(crd)
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end
end

        
        
    

generative (generic function with 2 methods)

#### Belief updater

$$b'(s_j)=P(s_j\vert o,a,b)=\dfrac{P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}{\sum_{s_j\in \mathcal{S}}P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}$$

In [14]:
# Input: prior belief, action, observation, index of the component
# Output: updated belief of the component
function update_belief(b,a,o,i) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    pomdp = Unit_pomdps[i]
    updated_belief = zeros(length(b))
    if o==4
        updated_belief[Number_level]=1.0
        return updated_belief
    end

    if (a==3) ||(a==4)
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: Number_level
            p_obs = Observation_matrix[sp,o]
            p_trans = pdf(transition(pomdp,1,1),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:length(states(pomdp)) 
        # p_observation = POMDPs.observation(pomdp,a,sp) 
        # p_obs = pdf(p_observation,o)
        p_obs = Observation_matrix[sp,o]
        p_trans = 0.0
        for s in 1:length(states(pomdp))
            #trans = transition(pomdp, s,a)
            #p_trans += pdf(transition(pomdp, s,a),sp) * pdf(b, s)
            # p_trans += T[s,sp,a] * pdf(b, s)
            # p_trans += T[s,sp,a,i] * b[s]
            p_trans += pdf(transition(pomdp,s,a),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end


update_belief (generic function with 1 method)

### Value Iteration

In [15]:
function ValueIteration(γ)
    # Here we always consider the state with the highest belief as the true state
    Q = zeros((Number_level, 4, NumberUnits))
    V = zeros(Float64, (Number_level, NumberUnits)).+1000
    for k in 1:NumberUnits
        for i in 1:10000
            for s in 1:Number_level
                Threads.@threads for a in 1:4
                    if a == 4
                        Q[s,a,k] = (reward(Unit_pomdps[k],s,a)) + γ*(sum(T[s,s1,3,k]*V[s1,k] for s1 in 1:Number_level))
                    else
                    Q[s,a,k] = (reward(Unit_pomdps[k],s,a)) + γ*(sum(T[s,s1,a,k]*V[s1,k] for s1 in 1:Number_level))
                    end
                end
                V[s,k] = findmax(Q[s, 1:4, k])[1]
            end
        end
    end
    return Q
end

ValueIteration (generic function with 1 method)

### System-level decision making

In [16]:
function decision(o, b)
    a_final = repeat(1:1,NumberUnits)
    # Instead of using maximum likelihood state,
    # we use belief-weighted sum of Q function

    failed_components = 0
    for i in 1:NumberUnits # numerate over units
        if o[i] == Number_level # failure state is self-announcing
            failed_components+=1
        end
    end

    # calculate belief-based Q table for each unit
    Q_belief = zeros((NumberUnits, 4))
    
    for i in 1:NumberUnits
        belief = b[i]
        for a in 1:4
            Q_belief[i,a] = sum(belief[s] * Q[s,a,i] for s in 1:Number_level)
        end
    end

    # decision making: 3 scenarios
    # 1) System fails
    if failed_components >= limit
        for i in 1:NumberUnits
            a_final[i]=argmax(a->Q_belief[i,a],[2,3])
            if o[i] == Number_level
                a_final[i]=4
            end
        end
        return a_final
    end

    q0 = 0.0
    a0 = repeat(1:1, NumberUnits)
    for i in 1:NumberUnits
        q0+=Q_belief[i,1]
    end

    q1 = 0.0
    a1 = repeat(1:1, NumberUnits)
    for i in 1:NumberUnits
        a1[i] = argmax(a->Q_belief[i,a], [2,3])
        if o[i] == Number_level
            a1[i] = 3
        end
        q1 += Q_belief[i, a1[i]]
    end

    if !(3 in a1)
        a1 = a0
        q1 = q0
    end

    if q0 > q1
        return a0
    else
        return a1
    end
end
    



decision (generic function with 1 method)

### Iteration

In [17]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])

cost=[ 
    [0	-1200	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2800   -100    -1200 -2000]
] 


for units in [20, 30, 40]

    for K in 4:4:convert(Int64, floor((units+1)/2))  

    global NumberUnits=convert(Int64,units)
    global limit=convert(Int64,K)
    #for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
    for lambda in 1:1:1
    global lamb=lambda
    for c in cost
        tick()
        println(units)  
        print("K : ")
        println(K)
        global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
        println(c)  
        global NumberUnits=units

# POMDPs are defined
global Unit_pomdps = create_pomdps()
# CONTINUE to obtain TRIVIAL SOLUTION and compute expected total rewards

# Obtain Q function using Value Iteration, same as CW-MDP
global Q = ValueIteration(gamma)
println(isnan(Q[1,1,1]))
if sum(isnan(Q[1,1,1])) > 0.0
    continue
end

tick()

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
    s=repeat(1:1,NumberUnits)
    o = rand(Initialobs, NumberUnits)
    belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
    R_sequence=[] 
    r=repeat(1:1,NumberUnits)*1.0

    # iteration
    for k in 1:simsteps
        a=decision(o,belief) # system-level decision making: action vector
        # SANITY CHECK
        # Update belief for each component individually
        #b=update_belief(b, a, o, multiunit) # what is the sequence of observing, updating belief, taking action?
        
        R_instant = 0.0
        global failed_components=0
        # When generating new state and new reward, we use true states
        for i in 1:NumberUnits # numerate over units
            if o[i]==Number_level # failure state is self-announcing
                failed_components+=1
            end
            s[i],r[i],o[i]=generative(s[i],a[i],i,234) #call generative function for underlying states
            # it might be better to use generative function to give o as well
            # o[i]=emission(s[i]) # observation generated by new underlying state
            belief[i]=update_belief(belief[i], a[i], o[i], i)
            # sum up single-component cost
            R_instant += r[i]
        end
        # add setup cost
        if (2 in a) ||(3 in a) ||(4 in a)
            R_instant += setup_cost
        end
        # add system failure penalty
        if failed_components >= limit
            R_instant += system_penalty
        end
        # record instant reward of current time step
        append!(R_sequence, R_instant)

    end
    R_discounted=0
    for k in 1:simsteps
        R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
    end
    results[h]=R_discounted
end # end the 10000 trials

println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()

                    
                    
    end
    end        
end
end


20
K : 4


[ Info:  started timer at: 2025-03-07T16:08:28.324


[0 -1200 -100 -1200 -2000]
false


[ Info:  started timer at: 2025-03-07T16:08:52.118


20
k : 4
1
-22148.67686171796
2117.485180883976
20
K : 4
[0 -2000 -100 -1200 -2000]


...

[ Info:          419.1910249s: 6 minutes, 59 seconds, 191 milliseconds
[ Info:  started timer at: 2025-03-07T19:29:29.227


false


[ Info:  started timer at: 2025-03-07T19:30:18.763


40
k : 20
1
-46292.85455619754
2826.234564572129


[ Info:          416.8679467s: 6 minutes, 56 seconds, 867 milliseconds


In [18]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,40.0,20.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,1.0,-46292.9,2826.23
2,40.0,20.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-41944.1,2729.44
3,40.0,20.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,1.0,-35998.5,2365.16
4,40.0,16.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,1.0,-46520.9,2875.0
5,40.0,16.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-42014.8,2681.33
6,40.0,16.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,1.0,-36032.8,2357.18
7,40.0,12.0,0.0,-2800.0,-100.0,-1200.0,-2000.0,1.0,-46874.2,2907.75
8,40.0,12.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-42161.6,2719.48
9,40.0,12.0,0.0,-1200.0,-100.0,-1200.0,-2000.0,1.0,-36063.9,2362.79


In [19]:
using CSV
CSV.write("Hetero CW-MDP.csv", df)

"Hetero CW-MDP.csv"